# JavaScript Rendered Diagramming

A variety of JavaScript based text to diagram tools exist that can be used to render diagrams from simple text based source scripts. Examples include:

- [`js-sequence-diagrams`](https://bramp.github.io/js-sequence-diagrams/);
- [`mermaid.js`](https://mermaid-js.github.io/mermaid/#/)
- [timing diagrams (`wavedrom`)](https://wavedrom.com/)

The [`innovationOUtside/nb_js_diagrammers`](https://github.com/innovationOUtside/nb_js_diagrammers) Python package provides a range of IPyhton magics for supporting the generation of diagrams using all these packages using simple cell block magics. 

```{note}
The [*Markdown preview enhanced*](https://shd101wyy.github.io/markdown-preview-enhanced/#/) VS Code extension provides a wide range of power tools to support the editing and previewing of native markdown documents. Among other things, the extension will offer rendered previews of diagrams from appropriate scripts using the `mermaid` and `wavedrom` JavaScript packages.

See also: [*Previewing Sphinx and Jupyter Book Rendered Mermaid and Wavedrom Diagrams in VS Code*](https://blog.ouseful.info/2021/11/02/previewing-sphinx-and-jupyter-book-rendered-mermaid-and-wavedrom-diagrams-in-vs-code/)
```

In [1]:
%%capture
try:
    import nb_js_diagrammers
except:
    %pip install nb_js_diagrammers

In [2]:
%load_ext nb_js_diagrammers

## Generating Timing Diagrams With `wavedrom`

In [3]:
%%wavedrom_magic -h 100 -o wavedrom.html

{ signal : [
  { name: "clk",  wave: "p......" },
  { name: "bus",  wave: "x.34.5x",   data: "head body tail" },
  { name: "wire", wave: "0.1..0." },
]}

## Generating Diagrams Using `mermaid.js`

Several diagram types can be generated using the `mermaid.js` package.

For example, we can generate simple flow charts:

In [4]:
%%mermaid_magic -h 500 -o mermaid-flowchart.html

flowchart TD
    A[Start] --> B{Is it?};
    B -->|Yes| C[OK];
    C --> D[Rethink];
    D --> B;
    B ---->|No| E[End];

More generally, we can also create simple graph diagrams:

In [5]:
%%mermaid_magic -h 250 -o mermaid-graph.html

graph TD;
    A-->B;
    A-->C;
    B-->D;
    C-->D;

We can also generate sequence diagrams:

In [6]:
%%mermaid_magic -h 350 -o mermaid-sequence.html

sequenceDiagram
    Alice->>John: Hello John, how are you?
    John-->>Alice: Great!
    Alice-)John: See you later!

And entity relationship diagrams:

In [7]:
%%mermaid_magic -h 330 -o mermaid-er.html

erDiagram
    CUSTOMER ||--o{ ORDER : places
    CUSTOMER {
        string name
        string custNumber
        string sector
    }
    ORDER {
        int orderNumber
        string deliveryAddress
    }

## Visualising Audio Files with `wavesurfer`

In [8]:
%wavesurfer_magic -o wavesurfer.html -f https://ia902606.us.archive.org/35/items/shortpoetry_047_librivox/song_cjrg_teasdale_64kb.mp3

## Flow Charts

As well as flow charts generated using `mermaid.js` we can also use the `flowchart.js` package.

In an interactive notebook, we can simply use the `%%flowchart_magic` block magic to render a flowchart from a flowchart definition.

```{note}
For an earlier attempt at embedding `flowchart.js` diagrams in notebooks as an `ipywidget`, see [`flowchart_js_jp_proxy_widget`](https://github.com/innovationOUtside/flowchart_js_jp_proxy_widget).
```

In [9]:
%%flowchart_magic -h 100 -o flowchart.html

st=>start: Start
e=>end: End
op1=>operation: Generate
op2=>parallel: Evaluate
st(right)->op1(right)->op2
op2(path1, top)->op1
op2(path2, right)->e

It is also possible to generate a flowchart based on the contents of a code cell (in an interactive notebook, we can do this directly using the `%%pyflowchart_magic` block magic):

In [10]:
%%pyflowchart_magic -h 800 -o pyflowchart.html -x
import time

def demo(msg='demo'):
    for i in range(10):
        print(f'{msg} loopcount is {i}')
        time.sleep(i)

The `-x / --execute` flag executes the code in the current notebook scope. *Without the flag, the code is not executed.*

Another, lett friendly approach, is to have a normal code cell:

In [11]:
import time

def demo(msg='demo'):
    for i in range(10):
        print(f'{msg} loopcount is {i}')
        time.sleep(i)

And then capture the code from this code cell and render it.

In [12]:
%%capture code
# This gets the content of the previous cell
# and stores it in the variable: code
%history -l 1

In [13]:
from pyflowchart import Flowchart

from nb_js_diagrammers.flowchartjs import TEMPLATE_FLOWCHARTJS
from nb_js_diagrammers.magics import js_ui

# Generate a flowchart from the grabbed code
fc = Flowchart.from_code(code.stdout)

# Render the flowchart
js_ui({"src":fc.flowchart()}, TEMPLATE_FLOWCHARTJS,
      out_fn='pyflowchart_grab.html', height=800)